# Client system operations agent system

Este proyecto realiza operaciones sobre los clientes de una empresa. 
En este caso, consulta los clientes de una empresa y genera un e-mail utilizando el idioma prefrido del cliente

1. Utiliza Langchain agents.
1. No requiere suscriciones, Utiliza solamente modelos de LLM locales y ligeros, en este caso usamos qwen3:7b sobre ollama.
1. No requiere conocimientos de IA tooling ya que utiliza el LLM local para generar su própio código python.
1. Utiliza el propio LLM local para generar los datos dummy con los que trabajan el resto de agentes.


In [ ]:
# %%capture --no-stderr
%pip install -U --upgrade pip langgraph langchain_community langchain_anthropic langchain-tavily langchain_experimental langchain_ollama mcp langchain-mcp-adapters langfuse pandas

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
workfolder = os.getenv('WORKFOLDER')
llm_config = {"recursion_limit": 10}

In [ ]:
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent

simple_agent = create_react_agent(name="simple_agent", model=ChatOllama(model="qwen3"), 
    tools=[], response_format='json', prompt="You are a helpful assistant. Your task is to respond to a user's question.")

input_message = {"role": "user", "content": """
Generate the python code for a function that writes a string content in a file name path given. 
the function is a langchain tool
three parameters: content , full_file_path and append (True or False)
should be able to handle exceptions and return an error message if the file cannot be written.
Function must have a docstring
use encoding utf8
give just the code, no introduction /no_think"""
}

# Use the agent
for step in simple_agent.stream(
    {"messages": [input_message]}, llm_config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

In [ ]:
from langchain_core.tools import tool 

@tool
def write_to_file(content: str, full_file_path: str, append: bool = False) -> str:
    """
    Writes the given content to a file at the specified full_file_path.
    If append is True, the content is added to the end of the file.
    If False, the file is overwritten.
    Returns a success message or an error message if writing fails.
    """
    try:
        mode = 'a' if append else 'w'
        with open(full_file_path, mode, encoding='utf-8') as file:
            file.write(content)
        return f"Successfully wrote content to {full_file_path}"
    except Exception as e:
        return f"Failed to write to file {full_file_path}. Error: {str(e)}"

In [ ]:
simple_agent = create_react_agent(name="simple_agent", model=ChatOllama(model="qwen3"), 
    tools=[write_to_file], response_format='json', prompt="You are a helpful assistant. Your task is to respond to a user's question.")

input_message = {"role": "user", "content": f"""
generate a csv list of 50 dummy clients
with columns headers 
with client_id, name, address, phone_number, email, prefered_languaje
write the list in a file in '{workfolder}/clients.csv'
 /no_think"""
}

# Use the agent
for step in simple_agent.stream(
    {"messages": [input_message]}, llm_config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

In [ ]:
input_message = {"role": "user", "content": f"""
Generate the python code for a function that returns the client details for a client id given. 
the function is a langchain tool
one parameters: client_id
use pandas to load a csv file in '{workfolder}/clients.csv'
find the client details in a pandas dataframe return dict with all columns
should be able to handle exceptions and return an error message if the file cannot be found.
Function must have a docstring
use encoding utf8
give just the code, no introduction /no_think"""
}

# Use the agent
for step in simple_agent.stream(
    {"messages": [input_message]}, llm_config, stream_mode="values"
):
    step["messages"][-1].pretty_print()


In [ ]:
import pandas as pd
from langchain_core.tools import tool 

@tool
def get_client_details(client_id):
    """
    Retrieve client details of the provided client ID.

    Parameters:
    client_id (str): The ID of the client to retrieve details for.

    Returns:
    dict: A dictionary containing all client details if found, otherwise an error message.
    """
    try:
        file_path = workfolder+'/clients.csv'
        df = pd.read_csv(file_path, encoding='utf-8')
        client_details = df[df['client_id'] == int(client_id)].to_dict(orient='records')
        if client_details:
            return client_details[0]
        else:
            return {"error": "Client not found"}
    except FileNotFoundError:
        return {"error": "File not found"}
    except Exception as e:
        return {"error": str(e)}

get_client_details.invoke('31')

In [ ]:
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent
import random

simple_agent = create_react_agent(name="simple_agent", model=ChatOllama(model="qwen3"), 
    tools=[get_client_details], response_format='json', prompt="You are a helpful assistant. Your task is to respond to a user's question.")

input_message = {"role": "user", "content": f"""
get the client details of a client with id '{random.randint(0, 50)}'
generate an email explaining we could not deliver the package at his address nor contact him at his phone number.
use they prefered language
especify the actual address and phone number of the client.
/no_think"""
}

# Use the agent
for step in simple_agent.stream(
    {"messages": [input_message]}, llm_config, stream_mode="values"
):
    step["messages"][-1].pretty_print()